In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

packageName='07-sklearn.ensemble.a-bagging'
classifierName='RandomForestClassifier'
extraParameterName='4-Random oversampling'

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

In [ ]:
# Hyperparameters Results

n_estimatorsFound=84
max_depthFound=32

resultMd.update_hyperparameter_config_result(packageName,classifierName,extraParameterName,max_depthFound)


print('done')

In [ ]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train, y_train = sp.randomOverSampling(x_train0, y_train0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()

modelClf = RandomForestClassifier(random_state=42)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')


fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)

# Hyperparameters tuning

In [ ]:
%%script false

from scipy.stats import randint

modelClf = RandomForestClassifier(random_state=42)
dic_param={
    'n_estimators': randint(40,110),
    'max_depth': randint(16,35)
}
res=fd.hyperparameterSelectionRandomizedSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train, iter=4)
print(res)

#{'max_depth': 27, 'n_estimators': 63}
#0.9642178429606372
#score   0.9903567984570878

#{'max_depth': 31, 'n_estimators': 93}
#0.9683656184340551
#score   0.9953890489913545

#{'max_depth': 31, 'n_estimators': 84}
#0.9687095771679202
#score   0.9953890489913545


In [ ]:
%%script false


modelClf = RandomForestClassifier(random_state=42)

dic_param={
    'n_estimators':[83,84,85],
    'max_depth':[30,31,32]
}
res=fd.hyperparameterSelectionGridSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)

#{'max_depth': 33, 'n_estimators': 85}
#0.9693208771667919
#scoref1 0.9955794733807419

#{'max_depth': 27, 'n_estimators': 60}
#0.9656079978532975
#scoref1 0.9899652643766885

#{'max_depth': 31, 'n_estimators': 92}
#0.9689044402825845
#scoref1 0.9953890489913545

#{'max_depth': 32, 'n_estimators': 84}
#0.9696553873218041
#scoref1 0.9953890489913545

# Final validation

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

 
modelClf = RandomForestClassifier(random_state=42)
parameters={'max_depth': max_depthFound, 'n_estimators':n_estimatorsFound}
modelClf.set_params(**parameters)
modelClf.fit(x_train, y_train)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)


In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
threshold=0.30
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    y_pred_prob = modelClf.predict_proba(dfTrx[predictors])[:,1]
    y_pred_adjusted = (y_pred_prob>threshold).astype(int)

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], y_pred_adjusted,'All', True)

    fd.show_confusion_matrix(dfTrx['Class'], y_pred_adjusted)

    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)

resultMd.update_performance_nextdays_result(packageName,classifierName,extraParameterName, f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])

